In [1]:
%load_ext autoreload

In [ ]:
"""
Subjects articulate 38 distinct phonemes that span the entire English language phonetic space in `silent' and `audible' manner. 

38 phonemes are (labels are given in brackets beside the phoneme)

Bilabial consonants: Baa (0), Paa (1), Maa (2)
Labiodental consonants: Faa (3), Vaa (4)
Dental consonants: Thaa (5), Dhaa (6)
Alvelor consonants: Taa (7), Daa (8), Naa (9), Saa (10), Zaa (11)
Post vaelor consonants: Chaa (12), Shaa (13), Jhaa (14), Zhaa (15)
Velar consonants: Kaa (!6), Gaa (17), NGaa (18)
Approximant consonants: Yaa (19), Raa (20), Laa (21), Waa (22)
Vowels:
OY as in bOY (23), OW as in nOW (24),
AO as in OUght (25), AA as in fAther (26),
AE as in At (27), EH as in mEt (28),
EY as in mAte (29), IY as in mEET (30),
IH as in It (31), AH as in HUt (32),
UW as in fOOD (33), ER as in hER (34),
UH as in hOOD (35)

DATA is given in a numpy array of dimensions (380, 22, 7500) - (38 phonemes each repeated 10 times, 22 channels, 7500 time samples).
Raw data was filtered using 3rd order Butterworth bandpass filter between 80 and 1000 Hertz.
"""

In [2]:
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from spdLearning import spdNN
from spdLearning import optimizers 
from spdLearning import trainTest
from spdLearning import spdNet

In [3]:
class BaseDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __getitem__(self, index):
        return self.data[index].astype('float32'), self.labels[index]

    def __len__(self):
        return len(self.data)

In [4]:
dev = "cpu" 
device = torch.device(dev)

In [5]:
numberPhonemes = 38
numberVowelPhonemes = 15
trialsPerPhoneme = 10
numberTrials = numberPhonemes * trialsPerPhoneme
numberChannels = 22
windowLength = 7500

In [6]:
subjectNumber = 1
articulationManner = "Voiced"
subject = "Subject" + str(subjectNumber)

In [7]:
DATA = np.load("Experiment1/Phoneme/" + articulationManner + subject + ".npy")
   
mean = np.mean(DATA, axis = -1)
std = np.std(DATA, axis = -1)
unvoicedDATA = (DATA - mean[..., np.newaxis])/(std[..., np.newaxis] + 1e-5)

phonemeMatrices = unvoicedDATA
labelsByPhonemes = np.array([[i] * trialsPerPhoneme for i in range(numberVowelPhonemes)]).reshape(numberVowelPhonemes * trialsPerPhoneme)

Indices =  {}
for i in range(numberVowelPhonemes):
    Indices[i] = []
for i in range(len(labelsByPhonemes)):
    Indices[labelsByPhonemes[i]].append(i)

covariancesLabels = np.zeros((numberVowelPhonemes, trialsPerPhoneme, numberChannels, numberChannels))
for i in range(numberVowelPhonemes):
    for j in range(trialsPerPhoneme):
        covariancesLabels[i, j] = 1/windowLength * (phonemeMatrices[230 + Indices[i][j]] @ phonemeMatrices[230 + Indices[i][j]].T)

trainFeatures = np.zeros((numberVowelPhonemes * 6, numberChannels, numberChannels))
trainLabels = np.zeros((numberVowelPhonemes * 6))
count = 0
for i in range(numberVowelPhonemes):
    trainFeatures[count:count + 3] = covariancesLabels[i, :3]
    trainFeatures[count + 3:count + 6] = covariancesLabels[i, 5:8]
    trainLabels[count:count + 6] = [i] * 6
    count += 6

testFeatures = np.zeros((numberVowelPhonemes * 4, numberChannels, numberChannels))
testLabels = np.zeros((numberVowelPhonemes * 4))
count = 0
for i in range(numberVowelPhonemes):
    testFeatures[count:count + 2] = covariancesLabels[i, 3:5]
    testFeatures[count + 2:count + 4] = covariancesLabels[i, 8:10]
    testLabels[count:count + 4] = [i] * 4
    count += 4

In [8]:
trainDataset = BaseDataset(trainFeatures, trainLabels)
testDataset = BaseDataset(testFeatures, testLabels)
trainDataloader = DataLoader(trainDataset, batch_size = 32, shuffle = True)
testDataloader = DataLoader(testDataset, batch_size = 32, shuffle = False)

In [9]:
numberEpochs = 1000

model = spdNet.learnSPDMatrices(numberVowelPhonemes).to(device)
numParams = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(numParams)

lossFunction = nn.CrossEntropyLoss()
spdOptimizer = optimizers.MixOptimizer(model.parameters(), lr = 0.05)

5099


In [10]:
maxValue = 0
for epoch in range(numberEpochs):
    trainLoss, trainAccuracy = trainTest.trainOperation(model, device, trainDataloader, spdOptimizer, lossFunction)
    testLoss, testAccuracy = trainTest.testOperation(model, device, testDataloader, lossFunction)
    if maxValue < testAccuracy:
        maxValue = testAccuracy
    print(f'Epoch: {epoch + 1}/{numberEpochs}, Training loss: {trainLoss:.4f}, Training accuracy: {trainAccuracy:.2f}%, Test loss: {testLoss:.4f}, Test accuracy: {testAccuracy:.2f}%')
print(maxValue)

Epoch: 1/1000, Training loss: 0.0923, Training accuracy: 4.44%, Test loss: 0.0902, Test accuracy: 8.33%
Epoch: 2/1000, Training loss: 0.0908, Training accuracy: 4.44%, Test loss: 0.0897, Test accuracy: 8.33%
Epoch: 3/1000, Training loss: 0.0906, Training accuracy: 6.67%, Test loss: 0.0892, Test accuracy: 18.33%
Epoch: 4/1000, Training loss: 0.0900, Training accuracy: 11.11%, Test loss: 0.0887, Test accuracy: 18.33%
Epoch: 5/1000, Training loss: 0.0900, Training accuracy: 15.56%, Test loss: 0.0884, Test accuracy: 11.67%
Epoch: 6/1000, Training loss: 0.0894, Training accuracy: 5.56%, Test loss: 0.0881, Test accuracy: 20.00%
Epoch: 7/1000, Training loss: 0.0897, Training accuracy: 6.67%, Test loss: 0.0877, Test accuracy: 13.33%
Epoch: 8/1000, Training loss: 0.0882, Training accuracy: 10.00%, Test loss: 0.0874, Test accuracy: 18.33%
Epoch: 9/1000, Training loss: 0.0881, Training accuracy: 16.67%, Test loss: 0.0870, Test accuracy: 21.67%
Epoch: 10/1000, Training loss: 0.0874, Training accur

In [11]:
print(maxValue)

73.33333333333333


In [12]:
%autoreload